<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [5]:
# Install spark

In [11]:
# hard‐reset pyspark to the Spark 3.0 line
# nuke whatever you have
!pip uninstall -y pyspark

# install the last 2.x release
!pip install pyspark==2.4.8

Found existing installation: pyspark 3.0.1
Uninstalling pyspark-3.0.1:
  Successfully uninstalled pyspark-3.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 MB 2.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 35.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-2.4.8-py2.py3-none-any.whl size=220864363 sha256=bae2529790591af563cc780e13f62b57915b136c1ea669efda401406054728be
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/7a/30/ee/5a852a877bb88fd5a6f75665d2544e107b914c2bad97181751
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9
    Uninstalling py4j-0.10.9:
      Successfully uninstalled py4j-0.10.9


In [7]:
# Start session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("ItFinallyWorks") \
        .getOrCreate()

spark.range(5).show()   # if you see 0-4, you’re good

25/05/20 22:49:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [4]:
# Load the search-terms CSV into a Spark DataFrame
import urllib.request, os

csv_url = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
           "IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv")
local_csv = "/home/jupyterlab/searchterms.csv"

In [5]:
# grab the file once; skip if it’s already there
if not os.path.exists(local_csv):
    urllib.request.urlretrieve(csv_url, local_csv)

In [6]:
# Load the csv into a spark dataframe

In [7]:
# now let Spark read the local copy
df = spark.read.csv(local_csv, header=True, inferSchema=True)
df.show(5)        # <-- screenshot top5rows.jpg

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [8]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)
rows, cols = df.count(), len(df.columns)
print("Rows:", rows, "Columns:", cols)    

Rows: 10000 Columns: 4


In [9]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)
df.show(5)     

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [10]:
df.printSchema()                            

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [11]:
gaming_hits = df.filter(df.searchterm == "gaming laptop").count()
print("“gaming laptop” count:", gaming_hits)   # ← screenshot or note the valu

“gaming laptop” count: 499


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [12]:
from pyspark.sql.functions import col

df.groupBy("searchterm")\
  .count()\
  .orderBy(col("count").desc())\
  .show(5)   

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [13]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
import urllib.request, tarfile, shutil, os
from pyspark.ml.regression import LinearRegressionModel

model_url  = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
              "IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz")
local_tar  = "/home/jupyterlab/model.tar.gz"
model_dir  = "/home/jupyterlab/sales_model"

In [14]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [15]:
# download & extract
if not os.path.exists(local_tar):
    urllib.request.urlretrieve(model_url, local_tar)
shutil.rmtree(model_dir, ignore_errors=True)
tarfile.open(local_tar).extractall(path=model_dir)

model = LinearRegressionModel.load(model_dir)
print("Model loaded from:", model_dir)    # ← screensho

Py4JJavaError: An error occurred while calling o88.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jupyterlab/sales_model/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:294)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:236)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:322)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrE

In [16]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [17]:
from pyspark.ml.feature import VectorAssembler

# make single-row DataFrame for 2021
test_df = spark.createDataFrame([(2021,)], ["year"])
vec_df  = VectorAssembler(inputCols=["year"], outputCol="features").transform(test_df)

forecast = model.transform(vec_df)
forecast.select("prediction").show()

NameError: name 'model' is not defined